In [ ]:
import piplates.DAQC2plate as DAQC2

In [51]:
def barrier_at_limit_check(openlimit, closelimit):
    """
    Checks if barrier is at or beyond limit and stops barrier if it is moving
    in a direction that would make it worse.
    :param float: openlimit = lowest voltage for open
    :param fload: closelimit = highest voltage for closed
    
    Returns
    =======
    True or False. If True also shuts down power to barrier
    """
    
    import piplates.DAQC2plate as DAQC2
    direction = 0 # -1 closing, 0 stopped, 1 openning.
    if (DAQC2.getDAC(0,0)>=2.5):
        direction = -1
    else:
        direction = 1
    position = DAQC2.getADC(0,0)-DAQC2.getADC(0,1)
    if (position >= closelimit) and (direction == -1):
        DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers
        return True
    if (position <= openlimit) and (direction == 1):
        DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers
        return True
    return False

In [54]:
def troughctl():
    """
    Will run as separate process taking in commands through a pipe and returning data
    on demand through a second pipe.
    Iteration 1, collects data into a fifo and watches barrier position.
    """
    import time
    import numpy as np
    from collections import deque
    from piplates import DAQC2plate as DAQC2
    
    # Limits must be manually set for each trough.
    openlimit = 0.005 # low voltage indicating at open limit
    closelimit = 7.78 # hight voltage indicating at close limit

    
    # TODO: drop a pid file in /tmp so that an always running deamon can shut off power
    #   to the barriers is this is not running.
    
    
    
    poshigh =[]
    poslow = []
    balhigh = []
    ballow = []
    thermhigh = []
    thermlow = []
    
    pos_V = deque(maxlen=5)
    pos_std = deque(maxlen=5)
    bal_V = deque(maxlen=5)
    bal_std = deque(maxlen=5)
    therm_V = deque(maxlen=5)
    therm_std = deque(maxlen=5)
    time_stamp = deque(maxlen=5)
    cmd_deque = deque()
    
    timedelta = 0.500 #seconds
    execute = True
    while execute:
        starttime = time.time()
        stopat = starttime + timedelta - 0.200 # leave 200 ms for communications and control
        while (time.time()< stopat):
            poslow.append(DAQC2.getADC(0, 1))
            poshigh.append(DAQC2.getADC(0, 0))
            ballow.append(DAQC2.getADC(0, 3))
            balhigh.append(DAQC2.getADC(0, 2))
            thermlow.append(DAQC2.getADC(0, 4))
            thermhigh.append(DAQC2.getADC(0,5))
        time_stamp.append((starttime+stopat)/2)
        # position
        ndata = len(poshigh)
        high = np.array(poshigh,dtype=np.float64)
        low = np.array(poslow,dtype=np.float64)
        vals = high-low
        avg = (np.sum(vals))/ndata
        stdev = np.std(vals, ddof=1, dtype=np.float64)
        stdev_avg = stdev / np.sqrt(float(ndata))
        pos_V.append(avg)
        pos_std.append(stdev_avg)
        # balance
        ndata = len(balhigh)
        high = np.array(balhigh,dtype=np.float64)
        low = np.array(ballow,dtype=np.float64)
        vals = high-low
        avg = (np.sum(vals))/ndata
        stdev = np.std(vals, ddof=1, dtype=np.float64)
        stdev_avg = stdev / np.sqrt(float(ndata))
        bal_V.append(avg)
        bal_std.append(stdev_avg)
        # thermistor
        ndata = len(thermhigh)
        high = np.array(thermhigh,dtype=np.float64)
        low = np.array(thermlow,dtype=np.float64)
        vals = high-low
        avg = (np.sum(vals))/ndata
        stdev = np.std(vals, ddof=1, dtype=np.float64)
        stdev_avg = stdev / np.sqrt(float(ndata))
        therm_V.append(avg)
        therm_std.append(stdev_avg)
        
        # Check barrier positions
        barrier_at_limit = barrier_at_limit_check()
        # Send warnings and error messages
        # Check command pipe and update command queue
        # Each command is a python list. 
        #    element 1: cmd name (string)
        #    element 2: single command parameter (number or string)
        # Execute commands in queue
        while len(cmd_deque)>0:
            cmd = cmd_deque.popleft()
            # execute the command
            if cmd[0] == 'Stop':
                DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers
            else if cmd[0] == 'Send':
                # send current contents of the data deques
            else if cmd[0] == 'Start':
                # start barriers moving using current direction and speed
            else if cmd[0] == 'Direction':
                # set the direction
            else if cmd[0] == 'Speed':
                # set the speed
            else if cmd[0] == 'MoveTo':
                # adjust direction if necessary
                # set the stop position
                # start the barriers
            else if cmd[0] == 'MotorCal':
                # calibrate the voltages for starting motor and speeds
            else if cmd[0] == 'ConstPi':
                # maintain a constant pressure
                # not yet implemented
            else if cmd[0] == 'Quit':
                # initiate shutdown sequence.
                # pass any necessary messages to back through pipe.
                # depower barriers
                DAQC2.clrDOUTbit(0,0) # switch off power/stop barriers                
                # stop collecting data/leave this while loop
                execute = False
        # Delay if have not used up all 200 ms
        used = time.time()-starttime
        if used < 0.495:
            time.sleep(0.495 - used)
            
    # TODO: remove .pid file from /tmp before closing. May want to wrap in error processing
    #  so that the .pid can be removed on at least some crashes.
    #  make sure barriers are depowered
    
    print(time.time())
    print('time\t\tposition\t\tbalance\t\tthermistor')
    for i in range(len(pos_V)):
        print(f'{time_stamp[i]:.3f}\t{pos_V[i]:.3f}+/-{pos_std[i]:.1e}\t{bal_V[i]:.3e}+/-{bal_std[i]:.1e}\t{therm_V[i]:.3f}+/-{therm_std[i]:.1e}') 

In [38]:
troughctl()

time		position		balance		thermistor
1636583778.718	5.355+/-0.000	-1.270e+00+/-1.8e-04	-3.157+/-2.1e-04
1636583779.024	5.356+/-0.000	-1.270e+00+/-1.3e-04	-3.157+/-1.4e-04
1636583779.330	5.355+/-0.000	-1.270e+00+/-1.0e-04	-3.157+/-1.2e-04
1636583779.635	5.355+/-0.000	-1.270e+00+/-8.8e-05	-3.157+/-1.1e-04
1636583779.938	5.355+/-0.000	-1.270e+00+/-8.1e-05	-3.157+/-9.3e-05
1636583780.242	5.355+/-0.000	-1.270e+00+/-7.2e-05	-3.157+/-8.5e-05
1636583780.547	5.355+/-0.000	-1.270e+00+/-7.1e-05	-3.157+/-7.8e-05
1636583780.851	5.355+/-0.000	-1.270e+00+/-6.8e-05	-3.157+/-7.4e-05
1636583781.156	5.355+/-0.000	-1.270e+00+/-6.5e-05	-3.157+/-6.9e-05
1636583781.458	5.355+/-0.000	-1.270e+00+/-6.3e-05	-3.157+/-6.6e-05


In [39]:
troughctl()

time		position		balance		thermistor
1636583793.913	5.355+/-0.000	-1.269e+00+/-1.9e-04	-3.156+/-2.0e-04
1636583794.216	5.355+/-0.000	-1.269e+00+/-1.3e-04	-3.156+/-1.3e-04
1636583794.517	5.355+/-0.000	-1.269e+00+/-1.0e-04	-3.156+/-1.1e-04
1636583794.820	5.355+/-0.000	-1.269e+00+/-8.6e-05	-3.156+/-1.0e-04
1636583795.122	5.355+/-0.000	-1.269e+00+/-8.0e-05	-3.156+/-8.9e-05
1636583795.424	5.355+/-0.000	-1.269e+00+/-7.0e-05	-3.156+/-8.1e-05
1636583795.727	5.355+/-0.000	-1.269e+00+/-6.6e-05	-3.156+/-7.7e-05
1636583796.030	5.355+/-0.000	-1.269e+00+/-6.3e-05	-3.156+/-7.5e-05
1636583796.333	5.355+/-0.000	-1.269e+00+/-6.0e-05	-3.156+/-7.2e-05
1636583796.640	5.355+/-0.000	-1.269e+00+/-6.0e-05	-3.156+/-6.8e-05


In [40]:
troughctl()

time		position		balance		thermistor
1636592016.853	5.355+/-0.000	-1.265e+00+/-1.5e-04	-3.137+/-1.6e-04
1636592017.157	5.355+/-0.000	-1.265e+00+/-1.1e-04	-3.137+/-1.3e-04
1636592017.459	5.355+/-0.000	-1.265e+00+/-1.0e-04	-3.137+/-1.1e-04
1636592017.761	5.355+/-0.000	-1.265e+00+/-8.6e-05	-3.137+/-9.7e-05
1636592018.062	5.355+/-0.000	-1.265e+00+/-8.0e-05	-3.137+/-8.9e-05
1636592018.363	5.355+/-0.000	-1.265e+00+/-7.3e-05	-3.137+/-8.3e-05
1636592018.665	5.355+/-0.000	-1.265e+00+/-6.6e-05	-3.137+/-7.8e-05
1636592018.970	5.355+/-0.000	-1.265e+00+/-6.2e-05	-3.137+/-7.3e-05
1636592019.276	5.355+/-0.000	-1.265e+00+/-5.9e-05	-3.137+/-6.9e-05
1636592019.579	5.355+/-0.000	-1.265e+00+/-5.5e-05	-3.137+/-6.5e-05


In [41]:
troughctl()

time		position		balance		thermistor
1636641616.796	5.364+/-0.000	-1.266e+00+/-1.9e-04	-3.181+/-2.0e-04
1636641617.101	5.364+/-0.000	-1.266e+00+/-1.3e-04	-3.181+/-1.4e-04
1636641617.404	5.364+/-0.000	-1.266e+00+/-9.8e-05	-3.181+/-1.2e-04
1636641617.707	5.364+/-0.000	-1.266e+00+/-8.5e-05	-3.180+/-1.0e-04
1636641618.011	5.364+/-0.000	-1.266e+00+/-7.3e-05	-3.181+/-9.0e-05
1636641618.315	5.364+/-0.000	-1.266e+00+/-6.5e-05	-3.180+/-8.5e-05
1636641618.618	5.364+/-0.000	-1.266e+00+/-6.2e-05	-3.180+/-7.7e-05
1636641618.921	5.364+/-0.000	-1.266e+00+/-5.8e-05	-3.180+/-7.0e-05
1636641619.227	5.364+/-0.000	-1.266e+00+/-5.5e-05	-3.180+/-6.6e-05
1636641619.528	5.364+/-0.000	-1.265e+00+/-5.2e-05	-3.180+/-6.3e-05


In [42]:
troughctl()

time		position		balance		thermistor
1636738303.327	5.352+/-0.000	-1.283e+00+/-1.9e-04	-3.372+/-2.0e-04
1636738303.633	5.352+/-0.000	-1.283e+00+/-1.4e-04	-3.372+/-1.5e-04
1636738303.939	5.352+/-0.000	-1.283e+00+/-1.1e-04	-3.372+/-1.2e-04
1636738304.246	5.352+/-0.000	-1.284e+00+/-1.1e-04	-3.372+/-1.1e-04
1636738304.549	5.352+/-0.000	-1.284e+00+/-1.1e-04	-3.372+/-9.1e-05
1636738304.856	5.352+/-0.000	-1.284e+00+/-9.9e-05	-3.372+/-8.3e-05
1636738305.161	5.352+/-0.000	-1.284e+00+/-9.0e-05	-3.372+/-7.5e-05
1636738305.465	5.352+/-0.000	-1.284e+00+/-8.6e-05	-3.372+/-6.9e-05
1636738305.767	5.352+/-0.000	-1.285e+00+/-8.2e-05	-3.372+/-6.5e-05
1636738306.071	5.352+/-0.000	-1.285e+00+/-7.7e-05	-3.372+/-6.1e-05


In [46]:
troughctl()

time		position		balance		thermistor
1636738430.279	5.352+/-2.1e-04	-1.282e+00+/-1.7e-04	-3.369+/-1.8e-04
1636738430.582	5.352+/-1.4e-04	-1.282e+00+/-1.2e-04	-3.369+/-1.3e-04
1636738430.887	5.352+/-1.1e-04	-1.282e+00+/-9.5e-05	-3.369+/-1.0e-04
1636738431.188	5.352+/-9.6e-05	-1.282e+00+/-8.4e-05	-3.369+/-9.2e-05
1636738431.492	5.352+/-8.6e-05	-1.282e+00+/-7.6e-05	-3.369+/-8.3e-05
1636738431.799	5.352+/-7.8e-05	-1.282e+00+/-7.2e-05	-3.369+/-7.6e-05
1636738432.103	5.352+/-7.4e-05	-1.282e+00+/-6.7e-05	-3.369+/-7.1e-05
1636738432.410	5.352+/-7.0e-05	-1.281e+00+/-6.2e-05	-3.369+/-6.6e-05
1636738432.716	5.352+/-6.5e-05	-1.281e+00+/-6.2e-05	-3.369+/-6.1e-05
1636738433.023	5.352+/-6.3e-05	-1.281e+00+/-5.9e-05	-3.369+/-5.8e-05


In [48]:
# using deque of maxlen=5
troughctl()

time		position		balance		thermistor
1636753165.083	5.352+/-8.6e-05	-1.273e+00+/-7.3e-05	-3.334+/-7.8e-05
1636753165.389	5.352+/-7.9e-05	-1.273e+00+/-6.7e-05	-3.334+/-7.2e-05
1636753165.694	5.352+/-7.4e-05	-1.273e+00+/-6.2e-05	-3.334+/-6.8e-05
1636753165.996	5.352+/-6.8e-05	-1.273e+00+/-5.7e-05	-3.334+/-6.4e-05
1636753166.302	5.352+/-6.5e-05	-1.273e+00+/-5.4e-05	-3.334+/-6.2e-05


In [55]:
troughctl()

1636754792.307978
time		position		balance		thermistor
1636754789.982	5.352+/-7.8e-05	-1.274e+00+/-7.8e-05	-3.371+/-7.3e-05
1636754790.477	5.352+/-7.1e-05	-1.274e+00+/-7.1e-05	-3.371+/-6.9e-05
1636754790.972	5.352+/-6.7e-05	-1.274e+00+/-6.5e-05	-3.371+/-6.5e-05
1636754791.467	5.352+/-6.4e-05	-1.274e+00+/-6.1e-05	-3.371+/-6.0e-05
1636754791.963	5.352+/-6.1e-05	-1.274e+00+/-5.7e-05	-3.371+/-5.7e-05
